In [1]:
import pandas as pd
import numpy as np
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords
from pythainlp import word_vector
import torch
from IPython.display import display

# Check Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ PyTorch is running on: {device}")

✅ PyTorch is running on: cuda


In [2]:
# 1. Load Thai2Vec
print("⏳ Loading Thai2Vec model (thai2fit_wv)...")
try:
    wv_wrapper = word_vector.WordVector(model_name="thai2fit_wv")
    wv = wv_wrapper.get_model() 
    print("✅ Thai2Vec Loaded successfully.")
except Exception as e:
    print(f"❌ Error loading model: {e}")

# 2. Load CSV
file_name = r"d:\year4\สหกิจ\prachatai_test.csv"
try:
    df = pd.read_csv(file_name)
    print(f"✅ CSV Loaded: {len(df)} records")
except FileNotFoundError:
    print(f"❌ ไม่พบไฟล์ที่: {file_name}")

⏳ Loading Thai2Vec model (thai2fit_wv)...
✅ Thai2Vec Loaded successfully.
✅ CSV Loaded: 6789 records


In [3]:
# Stopwords Setup
stop_words = set(thai_stopwords())
my_custom_stops = {' ', '\n', '\t', '“', '”', '(', ')', '[', ']', '-', '.', ',', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9'}
stop_words.update(my_custom_stops)

# --- ฟังก์ชันเปลี่ยนประโยคให้เป็น Matrix ของคำ และ คืนค่าคำที่ใช้ด้วย ---
def get_word_vectors_and_tokens(text):
    tokens = word_tokenize(str(text), engine='newmm')
    
    vecs = []
    kept_tokens = []

    for word in tokens:
        if word not in stop_words and word.strip() != '':
            try:
                # ลองดึง Vector จาก Thai2Fit
                vec = wv.get_vector(word) 
                vecs.append(vec)
                kept_tokens.append(word) # เก็บคำที่ผ่านการกรองแล้ว
            except:
                pass # ถ้าคำไม่มีใน Dict ก็ข้ามไป
    
    # ถ้าไม่มีคำไหนแปลงได้เลย
    if len(vecs) == 0:
        # คืนค่า Matrix ว่าง (size 0, 300) และ List ว่าง
        return np.zeros((0, wv.vector_size)), [] 
    
    # คืนค่า 2 อย่าง: (Matrix ของ Vector, List ของคำที่ตรงกัน)
    return np.array(vecs), kept_tokens

print("✅ Function ready.")

✅ Function ready.


In [4]:
# เลือกมาทดสอบแค่ 5 แถว
df_small = df.head(5).copy()

print("🔄 Converting text to Word Matrices and Tokens...")

# เรียกใช้ฟังก์ชัน และเก็บผลลัพธ์ลงตัวแปรชั่วคราว
results = df_small['body_text'].apply(get_word_vectors_and_tokens)

# แยกผลลัพธ์ออกเป็น 2 คอลัมน์
df_small['word_matrices'] = results.apply(lambda x: x[0]) # เก็บ Matrix (ตัวเลข)
df_small['final_tokens']  = results.apply(lambda x: x[1]) # เก็บคำ (Text)

print("✅ Processing Done.")
display(df_small[['title', 'final_tokens']].head(3)) # โชว์ตัวอย่างผลลัพธ์

🔄 Converting text to Word Matrices and Tokens...
✅ Processing Done.


,title,final_tokens
0,แฮคเกอร์ Anonymous ลั่นทำสงครามไซเบอร์ครั้งใหญ...,"[17, พ, ย, 2558, รายงาน, ประกาศสงคราม, หัวรุนแ..."
1,สตูดิโอจิบลิต้านสงคราม วิจารณ์การแก้รัฐธรรมนูญ...,"[ส.ส., ญี่ปุ่น, ต้องการ, แก้, รัฐธรรมนูญ, กองก..."
2,We need Safety Zone สมาคมเพื่อสันติภาพนำเดินรณ...,"[สมาคม, สันติภาพ, เครือข่าย, เด็ก, เยาวชน, ประ..."


In [5]:
# ===========================
# 🔍 เลือกดูข่าวที่ต้องการ (แก้เลขตรงนี้!)
idx = 0
# ===========================

if idx < len(df_small):
    matrix_news = df_small['word_matrices'].iloc[idx]
    tokens_news = df_small['final_tokens'].iloc[idx]

    print(f"--- 📰 ข่าวที่ {idx+1} ---")
    print(f"จำนวนคำที่เหลือหลังตัด Stopwords: {len(tokens_news)} คำ")
    print(f"ขนาดของ Matrix (คำ x 300): {matrix_news.shape}") 

    print("\n--- 📝 รายชื่อคำ (Tokens) ---")
    print(tokens_news[:50]) # โชว์ 50 คำแรก

    print("\n--- 🔢 ตัวอย่าง Vector ---")
    if len(matrix_news) > 0:
        print(f"คำว่า: '{tokens_news[0]}'") 
        print(f"Vector (20 ตัวแรก): {matrix_news[0][:20]}")
        print("...")
        print(f"\nคำว่า: '{tokens_news[1]}'") 
        print(f"Vector (20 ตัวแรก): {matrix_news[1][:20]}")
else:
    print("❌ Index เกินจำนวนข้อมูลที่มี")

--- 📰 ข่าวที่ 1 ---
จำนวนคำที่เหลือหลังตัด Stopwords: 59 คำ
ขนาดของ Matrix (คำ x 300): (59, 300)

--- 📝 รายชื่อคำ (Tokens) ---
['17', 'พ', 'ย', '2558', 'รายงาน', 'ประกาศสงคราม', 'หัวรุนแรง', 'ออกมา', 'ประกาศ', 'ผู้อยู่เบื้องหลัง', 'โจมตี', 'กรุง', 'ปารีส', 'คืน', 'ศุกร์', 'ที่ผ่านมา', 'ภาพ', 'คลิป', 'โฆษก', 'สวมหน้ากาก', 'สัญลักษณ์', 'ออกมา', 'อ่าน', 'แถลง', 'ภาษา', 'ฝรั่งเศส', 'ใจความ', 'โจมตี', 'กรุง', 'ปารีส', 'ทั่วโลก', 'ตามล่า', 'เหมือน', 'ทำ', 'ตอนที่', 'โจมตี', 'สำนักพิมพ์', 'ปฏิบัติการ', 'โจมตี', 'ความเสียใจ', 'ครอบครัว', 'สูญเสีย', 'เหตุการณ์', 'ประกาศสงคราม', 'โจมตี', 'สำนักพิมพ์', 'ฝรั่งเศส', 'ต้นปี', 'ที่ผ่านมา', 'อ้างว่า']

--- 🔢 ตัวอย่าง Vector ---
คำว่า: '17'
Vector (20 ตัวแรก): [ 2.17996e-01  4.87145e-01  7.85160e-02  2.91262e-01 -1.14988e-01
 -3.57300e-02  5.92545e-01  1.00819e-01 -1.36525e-01 -6.82950e-02
 -2.27718e-01 -1.90961e-01 -2.21790e-02  2.20321e-01  2.39987e-01
  1.36656e-01  1.90440e-02  3.32210e-02 -1.85260e-02  5.58000e-04]
...

คำว่า: 'พ'
Vector (20 ตัวแร